In [ ]:
import pickle
import concurrent.futures
import itertools
import time
import pandas as pd
from tqdm import tqdm
from collections import defaultdict

from bikewaysim.paths import config
from bikewaysim.impedance_calibration import stochastic_optimization
from bikewaysim import general_utils
from bikewaysim.routing import rustworkx_routing_funcs

In [ ]:
od_matrix = pd.read_csv(config['bikewaysim_fp']/'od_matrix.csv')
ods = list(set(zip(od_matrix['orig_N'],od_matrix['dest_N'])))

networks = [
    config['bikewaysim_fp']/'current_traveltime.pkl',
    config['bikewaysim_fp']/'current_impedance.pkl',
    config['bikewaysim_fp']/'future_impedance.pkl'
]

# split ods into groups of 1000 each
chunked_ods = general_utils.chunks(ods,1000)
tasks = [(x,y) for x,y in itertools.product(networks,chunked_ods)]

In [ ]:
test_od = (6836826951, 5424428379)

In [ ]:
tasks = [x for x in tasks if test_od in x[1]]

In [ ]:
from importlib import reload
reload(rustworkx_routing_funcs)

results = []
for task in tasks:
    network_fp = task[0]
    batch_ods = task[1]
    with network_fp.open('rb') as fh:
        turn_G = pickle.load(fh)
    final_ods, path_lengths, shortest_paths = rustworkx_routing_funcs.rx_shortest_paths(batch_ods,turn_G)

    batch_ods = [batch_ods[idx] for idx in final_ods]

    modeled_results = {(batch_od[0],batch_od[1]):{'length':path_length,'edge_list':shortest_path} for batch_od, path_length, shortest_path in zip(batch_ods,path_lengths,shortest_paths)}
    results.append((network_fp.stem, modeled_results))

In [ ]:
from collections import defaultdict
completed_tasks = defaultdict(dict)
for name, result in results:
    completed_tasks[name].update(result)
modeled_results = completed_tasks['future_impedance']

In [ ]:
completed_tasks['future_impedance'].keys()

In [ ]:
completed_tasks['future_impedance'][test_od]

In [ ]:
links, turns, length_dict, geo_dict, turn_G = rustworkx_routing_funcs.import_calibration_network(config)

In [ ]:
modeled_results.keys()

In [ ]:
test = 1153

In [ ]:
# import matched ODS (i.e. already have the correct network nodes)
od_matrix = pd.read_csv(config['bikewaysim_fp']/'od_matrix.csv')
start_N = od_matrix.loc[od_matrix['orig_taz'] == test,'orig_N'].unique()[0]#.item()
# ods = list(set(zip(od_matrix['orig_N'],od_matrix['dest_N'])))
start_N

In [ ]:
from bikewaysim.routing import route_utils
from shapely.ops import LineString
future_geo = {key: LineString(route_utils.get_route_line(item['edge_list'],geo_dict)) for key, item in modeled_results.items() if key[0]==start_N}
# future_geo = {key: route_utils.get_route_line(item['edge_list'],geo_dict) for key, item in future_results_dict.items()}
# shortest_geo = {key:route_utils.get_route_line(item['edge_list'],geo_dict) for key, item in shortest_results_dict.items()}

In [ ]:
import geopandas as gpd
gpd.GeoSeries(future_geo,crs=config['projected_crs_epsg']).explore()

In [ ]:

network_fp = task[0]
batch_ods = task[1]
with network_fp.open('rb') as fh:
    turn_G = pickle.load(fh)
final_ods, path_lengths, shortest_paths = rustworkx_routing_funcs.rx_shortest_paths(test_od,turn_G)

In [ ]:
test_od

In [ ]:
import rustworkx as rx
node_to_idx, idx_to_node = rustworkx_routing_funcs.rx_conversion_helpers(turn_G)


if isinstance(test_od,tuple): # if there is only one od provided, convert to list
    test_od2 = [test_od]

test_od2 = [(node_to_idx[start],node_to_idx[end]) for start, end in test_od2]
results_rx = [rx.dijkstra_shortest_paths(turn_G,start,end,weight_fn=float) for start, end in test_od2]

In [ ]:
test = iter(results_rx[0].values())

In [ ]:
len(list(dict(results_rx[0]).values())[0])

In [ ]:
[x for x in list(dict(results_rx[0]).values())[0]]

In [ ]:
next(test)

In [ ]:
next(iter(results_rx[0]))

In [ ]:
test_od

In [ ]:
[[idx_to_node[x] for x in i[1:-1] if isinstance(idx_to_node[x],tuple)] for sublist in results_rx for i in sublist.values()]

In [ ]:
results_rx

In [ ]:
shortest_paths

In [ ]:
set([x[0] for x in tasks])

In [ ]:
print(len(ods))

In [ ]:
len(set([y for x in tasks for y in x[1]]))

In [ ]:
# future network
network_fp = config['bikewaysim_fp']/'future_impedance.pkl'
with network_fp.open('rb') as fh:
    turn_G = pickle.load(fh)

In [ ]:
test = 1153

In [ ]:
# import matched ODS (i.e. already have the correct network nodes)
od_matrix = pd.read_csv(config['bikewaysim_fp']/'od_matrix.csv')
od_matrix = od_matrix[od_matrix['orig_taz'] == test]
ods = list(set(zip(od_matrix['orig_N'],od_matrix['dest_N'])))

In [ ]:
path_lengths, shortest_paths = rustworkx_routing_funcs.rx_shortest_paths(ods,turn_G)


In [ ]:
print(len(shortest_paths))
print(len(ods))
print(len(path_lengths))

In [ ]:
path_lengths[0]

In [ ]:
modeled_results = {(od[0],od[1]):{'length':path_length,'edge_list':shortest_path} for od, path_length, shortest_path in zip(ods,path_lengths,shortest_paths)}

In [ ]:
modeled_results[(6836826951, 5424428379)]

In [ ]:
links, turns, length_dict, geo_dict, turn_G = rustworkx_routing_funcs.import_calibration_network(config)

In [ ]:
from bikewaysim.routing import route_utils
from shapely.ops import LineString
future_geo = {key: LineString(route_utils.get_route_line(item['edge_list'],geo_dict)) for key, item in modeled_results.items()}
# future_geo = {key: route_utils.get_route_line(item['edge_list'],geo_dict) for key, item in future_results_dict.items()}
# shortest_geo = {key:route_utils.get_route_line(item['edge_list'],geo_dict) for key, item in shortest_results_dict.items()}

In [ ]:
import geopandas as gpd
gpd.GeoSeries(future_geo,crs=config['projected_crs_epsg']).explore()

In [ ]:
od_matrix['future_geo'] = [future_geo.get(tuple(x)) for x in od_matrix[['orig_N','dest_N']].values]

In [ ]:
test_trip = od_matrix.sample(1)['dest_N'].item()

In [ ]:
gdf = gpd.GeoDataFrame(od_matrix,geometry='future_geo',crs=config['projected_crs_epsg'])#.explore()
gdf[gdf['dest_N']==test_trip].explore()

In [ ]:
print(test_trip)